In [27]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer , CamembertForTokenClassification , TFAutoModelForTokenClassification , pipeline  , TFCamembertForTokenClassification , CamembertTokenizer
    

import re

from thai2transformers.tokenizers import (
    ThaiRobertaTokenizer, ThaiWordsNewmmTokenizer)

In [43]:
tokenizer_wc = CamembertTokenizer.from_pretrained('./fine_tune/finetuned-ner-tf_test')

ner_wc = pipeline(task='ner',
         tokenizer= tokenizer_wc,
         model = TFAutoModelForTokenClassification.from_pretrained('./fine_tune/finetuned-ner-tf_test'),
         ignore_labels=[], 
         grouped_entities=True)

ner_wc_ng = pipeline(task='ner',
tokenizer= tokenizer_wc,
model = TFAutoModelForTokenClassification.from_pretrained('./fine_tune/finetuned-ner-tf_test'),
ignore_labels=[], 
grouped_entities=False)


Some layers from the model checkpoint at ./fine_tune/finetuned-ner-tf_test were not used when initializing TFCamembertForTokenClassification: ['dropout_264']
- This IS expected if you are initializing TFCamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFCamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFCamembertForTokenClassification were not initialized from the model checkpoint at ./fine_tune/finetuned-ner-tf_test and are newly initialized: ['dropout_189']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some layers from the model checkpoint at ./fine_tune/f

In [44]:
tokenizer = ThaiWordsNewmmTokenizer.from_pretrained('./fine_tune/wangchanberta-base-wiki-newmm')

ner_st = pipeline(task='ner',
         tokenizer= tokenizer,
         model = CamembertForTokenClassification.from_pretrained('./fine_tune/finetune_ner_newmm_thainer'),
         ignore_labels=[], 
         grouped_entities=True)

ner_st_ng = pipeline(task='ner',
         tokenizer= tokenizer,
         model = CamembertForTokenClassification.from_pretrained('./fine_tune/finetune_ner_newmm_thainer'),
         ignore_labels=[], 
         grouped_entities=False)




In [15]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

In [16]:
y_true = [['O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
y_pred = [['O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]

In [17]:
f1_score(y_true, y_pred)

0.5

In [7]:
from datasets import load_dataset

dataset = load_dataset("thainer")

label_col = 'ner_tags'
dataset['train'] = dataset['train'].map(
lambda examples: {'ner_tags': [i if i not in [13, 26] else 27
                                for i in examples[label_col]]}
)
label_maps = {i: name for i, name in
      enumerate(dataset['train'].features[label_col].feature.names)}
      
label_names = dataset['train'].features[label_col].feature.names
num_labels = dataset['train'].features[label_col].feature.num_classes

Downloading: 5.09kB [00:00, 914kB/s]                    
Downloading: 2.89kB [00:00, 1.02MB/s]                   
Dataset thainer downloaded and prepared to /Users/chanon/.cache/huggingface/datasets/thainer/thainer/1.3.0/2e78d40dbb97228eca897367a8e1eff7b57f790909ad331edbb12a3484476ba6. Subsequent calls will reuse this data.
100%|██████████| 6348/6348 [00:02<00:00, 2437.56ex/s]


In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'ner_tags', 'pos_tags', 'tokens'],
        num_rows: 6348
    })
})

In [54]:
token_test = dataset['train'][50]['tokens']
ner_test = dataset['train'][50]['ner_tags']

In [60]:
st_tag = ner_st_ng(('').join(token_test))

In [59]:
wc_tag = ner_wc_ng(('').join(token_test))